In [154]:
import numpy as np
import pandas as pd

alco = pd.read_csv("./code/niaaa-report.csv")
# Riempimento valori mancanti e impostazione indice
alco = alco.fillna(method="ffill", axis=0).set_index(["State", "Year"])
alco.head()

Beer  Wine  Spirits
State   Year                     
Alabama 2009  1.20  0.22     0.58
        2008  1.24  0.22     0.58
        2007  1.24  0.22     0.56
        2006  1.22  0.21     0.55
        2005  1.18  0.20     0.53

In [155]:
# Creazione serie che vogliamo analizzare statisticamente
beer_seriesNY = alco.loc["New York"]["Beer"]
beer_seriesCA = alco.loc["California"]["Beer"]

In [156]:
# Correlazione consumo birra tra NY e CA
print(beer_seriesNY.corr(beer_seriesCA), beer_seriesCA.corr(beer_seriesNY ))

0.9709778539165477 0.9709778539165477


In [157]:
# Covarianza consumo birra tra CA e NY
print(beer_seriesNY.cov(beer_seriesCA), beer_seriesCA.cov(beer_seriesNY))

0.01743816287878787 0.01743816287878787


In [158]:
# Skewness di ogni serie
print(beer_seriesCA.skew(), [x.skew() for x in (beer_seriesCA, beer_seriesNY)], beer_seriesNY.skew())

0.16457291292976317 [0.16457291292976317, 0.32838100586311286] 0.32838100586311286


In [159]:
frameNY = alco.loc["New York"]
frameNY.head()

,Beer,Wine,Spirits
Year,,,
2009,0.91,0.46,0.69
2008,0.92,0.47,0.67
2007,0.92,0.46,0.68
2006,0.88,0.45,0.65
2005,0.90,0.43,0.64


In [160]:
frameNY.skew()

Beer       0.328381
Wine       0.127308
Spirits    0.656699
dtype: float64

In [161]:
frameNY.corr()

,Beer,Wine,Spirits
Beer,1.000000,0.470690,0.908969
Wine,0.470690,1.000000,0.611923
Spirits,0.908969,0.611923,1.000000


In [162]:
frameNY.cov()

,Beer,Wine,Spirits
Beer,0.016103,0.002872,0.026020
Wine,0.002872,0.002312,0.006638
Spirits,0.026020,0.006638,0.050888


In [163]:
pop = pd.read_csv("./code/co-est00int-tot.csv", index_col=["STNAME","CTYNAME"], usecols=['STNAME', 'CTYNAME','ESTIMATESBASE2000', 'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009', 'CENSUS2010POP',
       'POPESTIMATE2010'])
pop.head()

ESTIMATESBASE2000  POPESTIMATE2000  POPESTIMATE2001  \
STNAME  CTYNAME                                                               
Alabama Alabama                   4447207          4452173          4467634   
        Autauga County              43751            44021            44889   
        Baldwin County             140416           141342           144875   
        Barbour County              29042            29015            28863   
        Bibb County                 19856            19913            21028   

                        POPESTIMATE2002  POPESTIMATE2003  POPESTIMATE2004  \
STNAME  CTYNAME                                                             
Alabama Alabama                 4480089          4503491          4530729   
        Autauga County            45909            46800            48366   
        Baldwin County           147957           151509           156266   
        Barbour County            28653            28594            28287   
        Bibb County               21199            21399            21721   

                        POPESTIMATE2005  POPESTIMATE2006  POPESTIMATE2007  \
STNAME  CTYNAME                                                             
Alabama Alabama                 4569805          4628981          4672840   
        Autauga County            49676            51328            52405   
        Baldwin County           162183           168121           172404   
        Barbour County            28027            27861            27757   
        Bibb County               22042            22099            22438   

                        POPESTIMATE2008  POPESTIMATE2009  CENSUS2010POP  \
STNAME  CTYNAME                                                           
Alabama Alabama                 4718206          4757938        4779736   
        Autauga County            53277            54135          54571   
        Baldwin County           175827           179406         182265   
        Barbour County            27808            27657          27457   
        Bibb County               22705            22941          22915   

                        POPESTIMATE2010  
STNAME  CTYNAME                          
Alabama Alabama                 4785298  
        Autauga County            54632  
        Baldwin County           183195  
        Barbour County            27411  
        Bibb County               22867

In [164]:
# Scegli dati di New York rimuovendo dati non validi per 2000 e 2010
pop_seriesNY = pop.loc["New York"].loc["New York"][1:-2]
# Modifica l'indice da una stringa a un interno (l'anno)
pop_seriesNY.index = pop_seriesNY.index.str[-4:].astype(int)
# Cambia ordinamento dati da
pop_seriesNY

2000    19001780
2001    19082838
2002    19137800
2003    19175939
2004    19171567
2005    19132610
2006    19104631
2007    19132335
2008    19212436
2009    19307066
Name: New York, dtype: int64

In [165]:
# Calcola correlazione tra il consumo di alcol e la popolazione di New York
frameNY.loc[2009:2000].corrwith(pop_seriesNY)

Beer      -0.143813
Wine       0.589849
Spirits    0.675142
dtype: float64

In [166]:
# Controllo significatività della correlazione
from scipy.stats import pearsonr
# Allinea manualmente gli indici
pop_sorted = pop_seriesNY.sort_index(ascending=False)
alco_10 = alco.loc["New York"][:10]
alco_10

,Beer,Wine,Spirits
Year,,,
2009,0.91,0.46,0.69
2008,0.92,0.47,0.67
2007,0.92,0.46,0.68
2006,0.88,0.45,0.65
2005,0.90,0.43,0.64
2004,0.91,0.41,0.62
2003,0.93,0.40,0.61
2002,0.95,0.38,0.59
2001,0.96,0.37,0.56


In [167]:
# Per ogni colonna di alco_10 prendi serie consumo e popolazione e ritorna tupla (bev,) + tupla di pearsonr (r, p-value)
corrs = [(bev,) + pearsonr(alco_10[bev], pop_sorted) for bev in alco_10.columns]
corrs

[('Beer', -0.14381251680815543, 0.6918361386691005),
 ('Wine', 0.5898489836641843, 0.0726763414486624),
 ('Spirits', 0.6751416540344424, 0.032182433155739425)]

In [168]:
# Crea DataFrame da tupla indicando colonne e indice
pd.DataFrame(corrs, columns=("bev", "r", "p-value")).set_index("bev")

,r,p-value
bev,,
Beer,-0.143813,0.691836
Wine,0.589849,0.072676
Spirits,0.675142,0.032182


In [169]:
alco2009 = alco
alco2009

Beer  Wine  Spirits
State   Year                     
Alabama 2009  1.20  0.22     0.58
        2008  1.24  0.22     0.58
        2007  1.24  0.22     0.56
        2006  1.22  0.21     0.55
        2005  1.18  0.20     0.53
...            ...   ...      ...
Wyoming 1981  1.91  0.24     1.27
        1980  1.85  0.24     1.32
        1979  1.86  0.22     1.30
        1978  1.82  0.22     1.36
        1977  1.79  0.21     1.32

[1683 rows x 3 columns]